*Python Financial Analysis Notes*
<br>
第一章 利息积累及货币的时间价值
《金融与保险精算数学》，陈伟森2009.
<br>
** time value of money 货币的时间价值**
<br>
## Mento Carlo 方法的定义
1. Mento Carlo方法是一种随机模拟方法，使用随机数（或更常见的伪随机数）来解决很多计算问题的方法，
		将所求解的问题同一定的概率模型相联系，用计算机实现统计模拟或抽样，以获得问题的近似解。
<br>
		例如：求圆周率pi的近似值：
			正方形内部有一个相切的圆，它们的面积之比是pi/4.
			如果在正方形内部随机产生10000个点，计算它们与中心点的距离，从而判断是否落在圆的内部。
			如果这些点均匀分布，则圆周率 pi = 4 * res/n。
			其中， res表示落到圆内的点的个数，n表示总的点的个数。
<br>
2.Mento Carlo方法可以用于金融学和数值计算科学领域，在期权定价和风险管理上有重要的作用。
		如用蒙特卡洛模拟来估计欧式期权的价值：
            到期指数水平ST(i)  ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)，
                              z是一个服从标准正态分布的随机变量。
            内在价值hT(i)      hT = np.maximum(ST-K, 0)
            估算出：期权现值C0 C0 = np.exp(-r*T) * np.sum(hT) / I
                              I是一随机变量z的个数。

            优点：很容易处理高纬度问题；
            缺点：高计算。
<br>
ref: [蒙特卡洛模拟估计欧式看涨期权的价值](https://blog.csdn.net/u014281392/article/details/76202493)
### 例子1.计算欧式期权的现值的蒙特卡洛估算函数
<br>
```
def mentocarlo_eurex():
    """蒙特卡洛估值"""
    S0 = 100.   # 初始股票指数水平S0
    K = 105.    # 欧式看涨期权的行权价格K
    T = 1.0     # 到期时间T
    r = 0.05    # 固定无风险短期利率r
    sigma = 0.2 # 固定波动率sigma

    I = 100000  # number of simulations, I个伪随机数
    np.random.seed(1000)
    z = np.random.standard_normal(I)    # 一个标准正态分布随机变量
    # 所有到期指数水平ST(i)：一个随机变量
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    # 到期时期权的所有内在价值hT(i)
    hT = np.maximum(ST-K, 0)
    # 蒙特卡洛估算函数估计的期权现值C0
    C0 = np.exp(-r*T) * np.sum(hT) / I

    print('蒙特卡洛估算函数估计的 European Call Option 期权现值:\t{:5.3f}'.format(C0))

    # visualize
    fname = r'D:\workspace\pyfinanalysis\pyfin\data\stockprice.xlsx'
    xlsx_file = pd.ExcelFile(fname)
    djia = xlsx_file.parse('DJIA')
    print('DJIA examples:\n{}'.format(djia.tail()))

    # 对数收益率
    djia['Log_Ret'] = np.log(djia['Close'] / djia['Close'].shift(1))
    # 波动率：通过滚动窗的标准差计算
    djia['Volatility'] = djia['Log_Ret'].rolling(window=100, center=False).std()
    # in Ipython
    # get_ipython().magic('matplotlib inline')
    djia[['Close', 'Volatility']].plot(subplots=True, color='blue', figsize=(8, 6), grid=True)

    # result
    return C0
```
<br>
# 概念
<br>
- 本金
<br>
- 利息 rate
        年利率 annual rate interest
        单利 single-interest method,   a(t) = 1+rt,    t>=0
        复利 compound-interest method, a(t) = (1+r)^t, t>=0
        复利计算频率 frequency of compouding, 又叫利息支付频率 frequency of interest payment, m.
            用 r^(m) 表示1年计息 m 次的名义利率  nominal rate of interest payable m times a year.
                例如，一个存款账号以每年3%支付利息，利息以月计。
                => 3% 叫做 每年计息12次的年名义利率。m = 12, r^(m) = 5%.

            tm 是计息的时间段的数量，常用年数/月份数/月份数等等。
            tm 为整数的时候：
                利息积累函数 a(t) = (1 + r^(m)/m)tm, tm>=0
                总额总量函数 A(t) = A(0) * a(t)
            tm非整数的时候：
                （1）把时间 tm 计算为小数，用 a(t), A(t) 计算；
                （2）计算整数的 tm， 再用整数tm的总量作为本金、计算非整数的计息期数，两部分钱相加。
            例如，存1000元入存款账户，年名义利率为 3%，每月计息一次。两年半后其积累总量为多少？在该时段的利息总量为多少？
                解：投资期为两年半（30个月）。
                    积累总量 A(t) = A(0) a(t) = 1000 * (1+0.03/12)^30 = 1077.78 (元)
                    利息总量 a(t) = 1077.78 - 1000 = 77.78 (元)
                如果是按照季度计息呢？
                    积累总量 A(t) = A(0) a(t) = 1000 * (1+0.03/4)^10 = 1077.58 (元)
                    利息总量 a(t) = 1077.58 - 1000 = 77.58 (元)
                如果是本金100元，年名义利率为4%，投资25个月呢？ 25/3 = 8.33季度。
                    （1） 100 * (1+0.04/4)^8.33 = 108.64 (元)
                    （2） 24个月（8个季度）后的积累总量为：
                            100 * (1+0.04/4)^8 = 108.29 (元)
                          第 25个月后的累计函数为：
                            108.29 * (1+0.04/12) = 108.65 (元)
<br>
**在相同的年名义利率下，利息支付越频繁，最后的积累总量越大。同时积累总量存在极限。**
<br>
        连续复利计算 continuous compounding， 当 m->无穷大时，积累总量趋向于一个极限。
            用 rc 表示计息期为无限小的时间间隔下的名义利率，即m->无穷大时的名义利率，
            又称为连续复利利率 continuously compounded rate of interest。
                lim(1+rc/m)^m = e^rc, m->无穷大
                a(t) = lim(1+rc/m)^tm = (lim(1+rc/m)^m)^t = e^(rc*t), m->无穷大
<br>
        实际利率 effective rate of interest i(t):
            从 t-1 年到 t 年之间1年内得到的利息总量与 t-1 年年初的积累总量之比。
            i(t) = I(t) / A(t-1) = (A(t)-A(t-1)) / A(t-1) = (a(t)-a(t-1)) / a(t-1)

            <=> 对单利方法，               i(t) = r / (1+r(t-1))，  实际利率<名义利率；
                对按每年复利计算的复利方法，i(t) = r，               实际利率=名义利率；
                对每年计息m次的复利方法，   i(t) = (1+r^(m)/m)^m -1，实际利率>名义利率；
                对连续复利方法，            i(t) = e^rc -1，实际利率>名义利率；
<br>
**实际利率可参考用于投资；对超过1年的投资，其积累总量的计算可以以实际利率为基础，而不参考名义利率。**
<br>
**以上的贷款或投资、单利或者复利的计算中，利息在期末支付。贴现率针对的是利息需要预先支付。**
<br>
        贴现率 rate of discount， 用 d 表示。
            名义本金 nominal principal, face value.
            贴现金额 discount
            实际本金 effective principal:  实际本金 = 名义本金 * （1-贴现率）
            贴现票据期间：
                实际利率 i = d / (1-d), d = i / (1+i)
            每年m次计息，计算贴现率与年实际利率：
                等价的年名义利率 annualized equivalent nominal rate of interest:
                    r^(m) = m * d^(m) / (m-d^(m))
                年实际利率：
                    i = (1 + r^(m)/m)^m -1 = (1 + d^(m)/(m-d^(m)))^m -1
            <br>
            例如，6月期国库券按照8%的贴现率出售，
                6月期利息为：                              (0.08/2) / (1-0.08/2) = 4.17%
                等价的按半年复利计息的年名义利率为：        2 * 0.0417 = 8.34%
                年实际利率是：                             (1 + (0.08/2)/(1-0.08/2))^2 -1 = 8.51%
                在该贴现率下1单位国库券在3年之后的积累值为：(1+i)^3 = (1+0.0851)^3 = 1.28
<br>
**以上的单利或者复利的计算中，假定利率不变。如果利率变化，用利息强度的概念来计算。**
<br>
        利息强度 force of interest，theta(t): 每一时点利率看成是积累总量的百分比，它为t时积累总量的变化率。
            theta(t) = 1/a(t) * da(t)/dt = derivative_of_a(t)/a(t)
                     = d_ln_a(t) / dt
            => a(t) = exp(integral(theta(s) * ds)), s=0...t

            => 单利时的利息强度： theta(t) = r/(1+rt), t>=0
               复利时的利息强度： theta(t) = ln(1+i)，不随时间变化，利息强度为连续复利利率。
<br>
**单利时：利息强度作为积累总量的百分比，其每一时点的利率随时间递减。**
**复利时：利息强度不随时间变化，利息强度为连续复利利率。**
<br>
            例子1，一笔资金以5%的单利计息，另一笔资金按每年4%的复利计息。什么时候两笔资金的利息强度相等？
                    在此之后，哪一笔资金的利息强度更高？
                  解：在时刻t单利资金的利息强度为    theta(t) = 0.05/(1+0.05t)
                      在任何时刻复利资金的利息强度为 theta(t) = ln(1.04)
                          0.05/(1+0.05t) = ln(1.04)
                          => t = 5.4967
                      在此时点之后，单利资金的利息强度要低于复利资金的利息强度。
<br>
            例子2，如果一笔资金以利息强度 theta(t)=0.02t 进行积累，求2年后和5年后其等价的年实际利率：
                   解：积累函数 a(2) = exp(integral(0.02s * ds)) = exp(0.01 s^2), s=0...2 = e^0.04
                       等价的年实际利率i (1+i)^2 = e^0.04
                       => i = e^0.02 -1 = 2.02%

                       a(5) = exp(integral(0.02s * ds)) = exp(0.01 s^2), s=0...5 = e^0.25
                       (1+i)^5 = e^0.25
                       => i = e^0.05 -1 = 5.13%
<br>
- 积累总量 accumulated amount = 本金 + 利息
- 总量函数 amount function, A(t),    A(t) = A(0) * a(t)
- 积累函数 accumulated function, a(t)
<br>
- 终值 future value： 在实际利率i下，今天1单位投资在该年末将积累至(1+i)个单位，
        积累总量(1+i)叫1单位投资在年末的终值。
        t年后终值为 (1+i)^t.
<br>
- 现值 present value： 实际利率i下，今天投资的(1/(1+i))单位资金在该年末将积累至1单位资金，
        (1/(1+i))叫1年后支付1单位资金的现值。
        t年后要支付1单位资金的现值为 (1/(1+i))^t.
<br>
        例子1-16，一个人想在8年后得到100，000美元用以支付其儿子在海外的教育经费。
                  如果一项投资计划的年利率为8%，且按月计息，他今天必须投资多少钱？
                  解：实际利率：        (1+ 0.08/12)^12 -1 = 8.30%
                      今天需要投资的钱：100000 / (1+0.083)^8 = 52841.16 (元)
<br>
- 贴现因子 v: v = 1 / (1+i) , v为1年之后支付1单位资金的现值。
        v = 1 / (1+i)
        d = i / (1+i)
        => d = iv,
        => v = 1-d
<br>
- 现金流 C_j: 在时刻 j=0, 1, ..., n 时所发生的款项的值。
- 价值等式：  复利时：如果年实际利率为i，贴现因子为v，现金流为C_j,
                现值P = sum(C_j * v^j), j=0...n
                终值F = (1+i)^n * P = sum(C_j * (1+i)^(n-j)), j=0...n
<br>
        例子1-18，在年实际利率i下，经过多长时间初始本金能够翻倍？
                  解：因为对于 j>0, C_j=0, C_0 = 1, F = 2,
                        （1+i)^n = 2
                      => n = ln(2) / ln(1+i)
                         ln(2) = 0.6391,
                         n = 0.6391/i * i/ln(1+i)
                         为了得到n的近似解，将 i=8% 代入上式中右边的分数式中，
                         => n=· 0.6391/i * 1.0395 = 0.72/i
**72定律 rule of 72: 可以通过实际利率除0.72来近似求出n值，即初始本金翻倍的时间段。**
<br>



# numpy 中的金融函数
        参数：
        rate: 每一期的利率，数值或矩阵（M，）
        nper: 期数
        per:  目前处在第几个还款期
        pmt:  payment，每期的还款金额
        pv:   现值
        fv:   终值
        支出：金额为负，收入：金额为正。
        in npv, rate: 折现率，scalar数值。
        values: 现金流，正数为收入，负数为投资。第一个值必须时初始的投资，即必须时负数。
        finance_rate:  interest rate paid on the cash flows, scalar.
        reinvest_rate: interest rate received on the cash flows upon reinvestment.

<br>
- 现值 np.pv： 计算未来金额在现在的价值
        np.pv(rate, nper, pmt, fv=0.0, when='end')

        例如：年利率5%， 每月投入100， 需要投入多少本金才可以在10年后获得15692.93？
            np.pv(0.05/12, 12*10, -100, 15692.93)
            # output: -100
<br>
- 终值 np.fv： 计算未来的价值
        np.fv(rate, nper, pmt, pv[, when='end'])

        例如：年利率5%，存100，然后每个月存100，10年后可以得到多少？
            np.fv(0.05/12, 12*10, -100, -100)
            # output: 15692.93
<br>
- 内部收益率 internal rate of return: irr
        np.irr(values)

        例如，投资100，按固定的时间间隔取款 [39,59,55,20],
              假定最终值0，那么投资的100最终产出时 39+59+55+20=173.
              因为阶段性取款，收益率不是简单的做平均，而是如下计算：
                -100 + 39/(1+r) + 59/(1+r)^2 + 55/(1+r)^3 + 20/(1+r)^4 = 0

              np.irr([-100,39,59,55,20])
              round(np.irr([-100,39,59,55,20]), 2)
              # output: 0.28
<br>
- 修正内部收益率 mirr：指用不同的利率计算支出和收入时所得的内部收益率，
        同时考虑投资成本(finance_rate)和现金再投资收益率 (reinvestment_rate)。
        mirr = nth_squart_of(FV(postive cash flows, reinvestment rate) / -PV(negative cash flows, finance rate)) -1.
 Details refer to [Calculation of the MIRR](http://hysyeah.top/page/22/).

        np.mirr(values, finance_rate, reinvest_rate)
<br>
- 每期支付金额 np.pmt
        np.pmt(rate, nper, pv[, fv=0, when='end'])

        例如：月付，贷款200，000，15年还清，年利率为7.5%，每月付：
            np.pmt(0.075/12, 12*15, 200000)
            # output: -1854.02
<br>
- 定期付款期数 nper
        np.nper(rate, pmt, pv, fv=0, when='end')

        例如：如果每月150，年利率为7%，要还清8000的贷款，需要多少期？
            nper = np.nper(0.07/12, -150, 8000) # nper is np.ndarray
            print('{:.02f}'.format(nper))
            print(round(float(np.nper(0.07/12, -150, 8000)), 2))
            # output: 64.07
<br>
- 每期支付金额之本金 ppmt
        np.ppmt(rate, per, nper, pv[, fv, when])
        在等额还款方式下，虽然每期偿还的总金额（本金+利息）相等，但是各期的本金及利息不相等，
        随着时间的推移，每期偿还的本金逐渐增加，利息逐渐减少。
<br>
- 每期支付金额之利息 ipmt
        np.ipmt(rate, per, nper, pv[, fv, when])
<br>
**pmt = ppmt + ipmt 每期支付金额 = 每期支付金额之本金 + 每期支付金额之利息**
<br>
- 利率 rate
        np.rate(nper, pmt, pv, fv[, when='end', guess=0.1, tol=1e-06, maxiter=100])
<br>
- 净现值 npv，指投资方案所产生的现金净流量（流入-流出）以资金成本为贴现率折现之后与原始投资额现值的差额。
        即：一项投资所产生的未来现金流的现值与项目投资成本的现值之间的差值。
        npv = sum((CI-CO) (1+i)^(-t))
               CI: 现金流入；CO：现金流出；(CI-CO)：第t年净现金流量；i：基准折现率
        npv(rate, values)

        NPV>0 表示项目实施以后，除保证可实现预定的收益率外，尚可获得更高的收益；
        NPV=0 表示。。。。。。，投资收益率正好达到预期，而不是投资项目盈亏平衡；
        NPV<0 表示。。。。。。，未能达到预定的收益率水平，而不能确定项目已亏损。
<br>


- ref: [Python数据分析_Numpy中的金融函数](https://www.jianshu.com/p/9ad131856078)

